# ⚡ Apache Spark Introduction

This notebook demonstrates Apache Spark fundamentals in the Big Data environment.

## Learning Objectives
- Initialize Spark session with Hadoop integration
- Work with Spark DataFrames and RDDs
- Perform data processing and transformations
- Save results to HDFS

## 1. Spark Session Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json

print('🚀 Big Data Environment - Apache Spark Introduction')
print('=' * 55)

# Create Spark session with Hadoop integration
spark = SparkSession.builder \
    .appName("BigDataEnv-SparkIntro") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

print('✅ Spark Session created successfully')
print(f'📊 Spark Version: {spark.version}')
print(f'🌐 Spark Master: {spark.sparkContext.master}')
print(f'📱 Application ID: {spark.sparkContext.applicationId}')
print(f'🔧 Hadoop Version: {spark.sparkContext._jsc.hadoopConfiguration().get("hadoop.version")}')

## 2. Working with Spark DataFrames

In [ ]:
# Read CSV file from HDFS
print('📖 Reading users data from HDFS...')

try:
    # Read users CSV file
    users_df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("hdfs://namenode:9000/user/demo/input/users.csv")
    
    print('✅ Users data loaded successfully')
    print(f'📊 Records: {users_df.count()}')
    print(f'🔢 Columns: {len(users_df.columns)}')
    
    # Show schema
    print('\n📋 Schema:')
    users_df.printSchema()
    
    # Show first few records
    print('\n👀 First 5 records:')
    users_df.show(5)
    
except Exception as e:
    print(f'❌ Error reading users data: {e}')
    # Create sample data if file doesn't exist
    sample_data = [
        (1, "John Smith", "john@email.com", 28, "New York", "USA"),
        (2, "Emma Johnson", "emma@email.com", 34, "London", "UK"),
        (3, "Michael Chen", "michael@email.com", 22, "Toronto", "Canada")
    ]
    schema = ["user_id", "name", "email", "age", "city", "country"]
    users_df = spark.createDataFrame(sample_data, schema)
    print('ℹ️  Using sample data instead')

## 3. Data Transformations and Analysis

In [ ]:
# Basic DataFrame operations
print('🔍 Data Analysis:')

# Count by country
print('\n📊 Users by Country:')
country_counts = users_df.groupBy("country") \
    .count() \
    .orderBy(col("count").desc())
country_counts.show()

# Age statistics
print('\n📈 Age Statistics:')
age_stats = users_df.select(
    avg("age").alias("avg_age"),
    min("age").alias("min_age"),
    max("age").alias("max_age"),
    stddev("age").alias("stddev_age")
)
age_stats.show()

# Filter users by age
print('\n👥 Users above 30:')
mature_users = users_df.filter(col("age") > 30) \
    .select("name", "age", "city", "country")
mature_users.show()

In [ ]:
# Advanced transformations
print('🔄 Advanced Transformations:')

# Add age group column
users_with_age_group = users_df.withColumn(
    "age_group",
    when(col("age") < 25, "Young")
    .when(col("age") < 35, "Adult")
    .otherwise("Mature")
)

print('\n👶 Age Group Distribution:')
age_group_dist = users_with_age_group.groupBy("age_group") \
    .count() \
    .orderBy("age_group")
age_group_dist.show()

# Create email domain analysis
users_with_domain = users_df.withColumn(
    "email_domain",
    split(col("email"), "@").getItem(1)
)

print('\n📧 Email Domain Analysis:')
domain_analysis = users_with_domain.groupBy("email_domain") \
    .count() \
    .orderBy(col("count").desc())
domain_analysis.show(10)

## 4. Working with JSON Data

In [ ]:
# Read JSON data from HDFS
print('📖 Reading transactions JSON from HDFS...')

try:
    # Read transactions JSON file
    transactions_df = spark.read \
        .option("multiline", "true") \
        .json("hdfs://namenode:9000/user/demo/input/transactions.json")
    
    print('✅ Transactions data loaded successfully')
    print(f'📊 Records: {transactions_df.count()}')
    
    # Show schema
    print('\n📋 Schema:')
    transactions_df.printSchema()
    
    # Show sample data
    print('\n💳 Sample Transactions:')
    transactions_df.select("transaction_id", "user_id", "amount", "currency", "merchant", "status").show(5)
    
except Exception as e:
    print(f'❌ Error reading transactions: {e}')
    # Create sample transactions data
    sample_transactions = [
        ("TXN001", 1, 150.50, "USD", "Amazon", "completed"),
        ("TXN002", 2, 89.99, "GBP", "Tesco", "completed"),
        ("TXN003", 1, 45.75, "USD", "Starbucks", "completed")
    ]
    schema = ["transaction_id", "user_id", "amount", "currency", "merchant", "status"]
    transactions_df = spark.createDataFrame(sample_transactions, schema)
    print('ℹ️  Using sample transactions data')

In [ ]:
# Analyze transaction data
print('💰 Transaction Analysis:')

# Total amount by currency
print('\n💱 Total Amount by Currency:')
currency_totals = transactions_df.groupBy("currency") \
    .agg(
        sum("amount").alias("total_amount"),
        count("*").alias("transaction_count"),
        avg("amount").alias("avg_amount")
    ) \
    .orderBy(col("total_amount").desc())
currency_totals.show()

# Transaction status distribution
print('\n📊 Transaction Status Distribution:')
status_dist = transactions_df.groupBy("status") \
    .count() \
    .orderBy(col("count").desc())
status_dist.show()

# Top merchants
print('\n🏪 Top Merchants by Transaction Count:')
top_merchants = transactions_df.groupBy("merchant") \
    .agg(
        count("*").alias("transaction_count"),
        sum("amount").alias("total_revenue")
    ) \
    .orderBy(col("transaction_count").desc())
top_merchants.show(10)

## 5. Joining DataFrames

In [ ]:
# Join users and transactions
print('🔗 Joining Users and Transactions:')

# Join on user_id
user_transactions = users_df.join(
    transactions_df,
    users_df.user_id == transactions_df.user_id,
    "inner"
).select(
    users_df.user_id,
    users_df.name,
    users_df.country,
    transactions_df.transaction_id,
    transactions_df.amount,
    transactions_df.currency,
    transactions_df.merchant,
    transactions_df.status
)

print('\n👥 User Transaction Details:')
user_transactions.show(10)

# Aggregate by user
print('\n💰 User Spending Summary:')
user_spending = user_transactions.groupBy(
    "user_id", "name", "country"
).agg(
    count("transaction_id").alias("total_transactions"),
    sum("amount").alias("total_spent"),
    avg("amount").alias("avg_transaction_amount")
).orderBy(col("total_spent").desc())

user_spending.show()

## 6. Working with RDDs

In [ ]:
# Convert DataFrame to RDD for low-level operations
print('🔧 Working with RDDs:')

# Convert users DataFrame to RDD
users_rdd = users_df.rdd

print(f'📊 RDD Partitions: {users_rdd.getNumPartitions()}')
print(f'📊 RDD Count: {users_rdd.count()}')

# Map operation: extract country names
countries_rdd = users_rdd.map(lambda row: row.country)
unique_countries = countries_rdd.distinct().collect()

print(f'\n🌍 Unique Countries: {unique_countries}')

# Filter operation: users over 30
mature_users_rdd = users_rdd.filter(lambda row: row.age > 30)
mature_count = mature_users_rdd.count()

print(f'\n👥 Users over 30: {mature_count}')

# Reduce operation: sum of all ages
total_age = users_rdd.map(lambda row: row.age).reduce(lambda a, b: a + b)
avg_age = total_age / users_rdd.count()

print(f'\n📈 Average Age (via RDD): {avg_age:.1f} years')

## 7. Spark SQL

In [ ]:
# Register DataFrames as temporary views
print('🗄️  Creating Temporary Views for Spark SQL:')

users_df.createOrReplaceTempView("users")
transactions_df.createOrReplaceTempView("transactions")

print('✅ Temporary views created: users, transactions')

# SQL Query 1: User demographics
print('\n📊 SQL Query: User Demographics by Country')
demographics_sql = """
SELECT 
    country,
    COUNT(*) as user_count,
    AVG(age) as avg_age,
    MIN(age) as min_age,
    MAX(age) as max_age
FROM users 
GROUP BY country 
ORDER BY user_count DESC
"""

spark.sql(demographics_sql).show()

# SQL Query 2: User transaction summary
print('\n💳 SQL Query: User Transaction Summary')
transaction_summary_sql = """
SELECT 
    u.name,
    u.country,
    COUNT(t.transaction_id) as total_transactions,
    SUM(t.amount) as total_spent,
    AVG(t.amount) as avg_transaction
FROM users u
JOIN transactions t ON u.user_id = t.user_id
WHERE t.status = 'completed'
GROUP BY u.user_id, u.name, u.country
ORDER BY total_spent DESC
"""

spark.sql(transaction_summary_sql).show()

## 8. Save Results to HDFS

In [ ]:
# Save processed data to HDFS
print('💾 Saving Results to HDFS:')

try:
    # Save user spending summary as Parquet
    output_path = "hdfs://namenode:9000/user/demo/output/user_spending_summary"
    
    user_spending.coalesce(1) \
        .write \
        .mode("overwrite") \
        .parquet(output_path)
    
    print(f'✅ User spending summary saved to: {output_path}')
    
    # Save country demographics as JSON
    demographics_path = "hdfs://namenode:9000/user/demo/output/country_demographics"
    
    country_demographics = spark.sql(demographics_sql)
    country_demographics.coalesce(1) \
        .write \
        .mode("overwrite") \
        .json(demographics_path)
    
    print(f'✅ Country demographics saved to: {demographics_path}')
    
    # Save as CSV too
    csv_path = "hdfs://namenode:9000/user/demo/output/user_spending_csv"
    
    user_spending.coalesce(1) \
        .write \
        .mode("overwrite") \
        .option("header", "true") \
        .csv(csv_path)
    
    print(f'✅ User spending CSV saved to: {csv_path}')
    
except Exception as e:
    print(f'❌ Error saving to HDFS: {e}')

## 9. Performance Monitoring

In [ ]:
# Monitor Spark application performance
print('📊 Spark Application Performance:')

# Get application info
sc = spark.sparkContext
print(f'📱 Application Name: {sc.appName}')
print(f'🆔 Application ID: {sc.applicationId}')
print(f'🌐 Master: {sc.master}')
print(f'⚙️  Default Parallelism: {sc.defaultParallelism}')

# Show execution plan for a complex query
print('\n🔍 Query Execution Plan:')
complex_query = user_spending.filter(col("total_spent") > 100)
complex_query.explain(True)

print('\n💡 Access Spark UI at: http://localhost:4040')
print('💡 Access Spark Master UI at: http://localhost:8080')

## 10. Cleanup

In [ ]:
# Stop Spark session (optional - typically done at the end)
print('🧹 Spark Session Cleanup:')
print('💡 Spark session will remain active for other notebooks')
print('💡 To stop the session, uncomment the line below:')
# spark.stop()

print('\n📊 Session still active - you can continue with other operations')

## 🎯 Summary

In this notebook, you learned:

1. **Spark Session**: How to create and configure a Spark session with Hadoop integration
2. **DataFrames**: Reading, transforming, and analyzing data with Spark DataFrames
3. **JSON Processing**: Working with JSON data and schema inference
4. **Data Joins**: Combining multiple datasets using DataFrame joins
5. **RDD Operations**: Low-level data processing with Resilient Distributed Datasets
6. **Spark SQL**: Using SQL syntax for data analysis
7. **Data Persistence**: Saving results to HDFS in various formats
8. **Performance Monitoring**: Understanding Spark execution and optimization

### Key Spark Concepts Covered
- **Transformations**: `select()`, `filter()`, `groupBy()`, `join()`, `withColumn()`
- **Actions**: `show()`, `count()`, `collect()`, `write()`
- **SQL Functions**: `avg()`, `sum()`, `min()`, `max()`, `when()`, `split()`
- **File Formats**: CSV, JSON, Parquet

### Next Steps
- Explore the **03-hive-sql.ipynb** notebook to learn Hive integration
- Check out the Spark Master UI at http://localhost:8080
- Browse the Spark Application UI at http://localhost:4040

### 🔗 Useful Links
- **Spark Master UI**: http://localhost:8080
- **Spark Application UI**: http://localhost:4040
- **Spark Documentation**: https://spark.apache.org/docs/latest/
- **PySpark API**: https://spark.apache.org/docs/latest/api/python/